In [1]:
import glfw
import numpy as np
from math import sin, cos
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
from OpenGL.GLU import *
from OpenGL.GLUT import *
from plyfile import PlyData, PlyElement
import pyrr
import math
from PIL import Image

In [2]:
vertex_src = """
# version 330

layout(location = 0) in vec3 a_position;
layout(location = 1) in vec3 a_color;

uniform mat4 model;
uniform mat4 projection;
uniform mat4 view;

out vec3 v_color;

void main()
{
    gl_Position = projection * view * model * vec4(a_position, 1.0);
    v_color = a_color;
}
"""

fragment_src = """
# version 330

in vec3 v_color;
out vec4 out_color;

void main()
{
    out_color = vec4(v_color, 1.0);
}
"""

In [3]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/male_face_c10s10e5.ply', 'rb') as f:
    face = PlyData.read(f)
#顶点 x y z 色彩 R G B A 数值为1
face_vertices=[]
for i in range(len(face.elements[0])):
    face_vertices.append(face.elements[0].data[i][0])
    face_vertices.append(face.elements[0].data[i][1])
    face_vertices.append(face.elements[0].data[i][2])
    face_vertices.append(face.elements[0].data[i][3])
    face_vertices.append(face.elements[0].data[i][4])
    face_vertices.append(face.elements[0].data[i][5])
face_vertices = np.array(face_vertices, dtype=np.float32)
#归一化，因vertex src的输入为0~1的数值
Min = np.min(face_vertices)
Max = np.max(face_vertices)
face_vertices = (face_vertices - Min) / (Max - Min)
#顶点索引，三点一组
face_indices=[]
for i in range(len(face.elements[1])):
    face_indices.append(face.elements[1].data[i][0][0])
    face_indices.append(face.elements[1].data[i][0][1])
    face_indices.append(face.elements[1].data[i][0][2])
face_indices = np.array(face_indices, dtype=np.uint32)

In [4]:
with open('c:/Users/alpha02/Documents/PLY-FACE-DATA/sphere_without_color.ply', 'rb') as s:
    sphere = PlyData.read(s)
#顶点 x y z 色彩 R G B  取颜色为蓝色 0 170 255 归一化 0 0.67 1
sphere_vertices = []
for i in range(len(sphere.elements[0])):
    sphere_vertices.append(sphere.elements[0].data[i][0]*0.2)
    sphere_vertices.append(sphere.elements[0].data[i][1]*0.2)
    sphere_vertices.append(sphere.elements[0].data[i][2]*0.2)
    sphere_vertices.append(0)
    sphere_vertices.append(0.67)
    sphere_vertices.append(1)
sphere_vertices = np.array(sphere_vertices, dtype=np.float32)
#顶点索引，三点一组
sphere_indices = []
for i in range(len(sphere.elements[1])):
    sphere_indices.append(sphere.elements[1].data[i][0][0])
    sphere_indices.append(sphere.elements[1].data[i][0][1])
    sphere_indices.append(sphere.elements[1].data[i][0][2])
sphere_indices = np.array(sphere_indices, dtype=np.uint32)    

In [41]:
def window_resize(window, width, height):
    glViewport(0, 0, width, height) 
# initializing glfw library
if not glfw.init():
    raise Exception("glfw can not be initialized!")
# creating the window
window = glfw.create_window(1024,1024, "My OpenGL window", None, None)
# check if window was created
if not window:
    glfw.terminate()
    raise Exception("glfw window can not be created!")
# set window's position
glfw.set_window_pos(window, 200, 200)
# set the callback function for window resize
glfw.set_window_size_callback(window, window_resize)
# make the context current
glfw.make_context_current(window)

#initial shader
shader = compileProgram(compileShader(vertex_src, GL_VERTEX_SHADER),compileShader(fragment_src, GL_FRAGMENT_SHADER))
glUseProgram(shader)
glClearColor(0, 0.1, 0.1, 1)
# Vertex Buffer Object 为物体分配顶点对象缓存 存储所有顶点信息，但不包含索引信息。VAO中包含VBO和EBO
VBO = glGenBuffers(2)
# Element Buffer Object 为物体索引信息分配缓存 只存储索引信息每个物体一个缓存
EBO = glGenBuffers(2)
view_loc = glGetUniformLocation(shader, "view")
view = pyrr.matrix44.create_look_at(pyrr.Vector3([0, 0, 3]), pyrr.Vector3([0, 0, 0]), pyrr.Vector3([0, 1, 0]))#设置相机,相机位置，相机指向位置，相机顶朝向
glUniformMatrix4fv(view_loc, 1, GL_FALSE, view)
proj_loc = glGetUniformLocation(shader, "projection")
projection = pyrr.matrix44.create_perspective_projection_matrix(100, 1024/1024, 0.1, 100)#第一个值设置大小，值越大物体越小
glUniformMatrix4fv(proj_loc, 1, GL_FALSE, projection)
model_loc = glGetUniformLocation(shader, "model")
glEnable(GL_DEPTH_TEST)
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
# the main application loop
while not glfw.window_should_close(window):
    glfw.poll_events()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    #载入面孔
    glBindBuffer(GL_ARRAY_BUFFER, VBO[0])
    glBufferData(GL_ARRAY_BUFFER, face_vertices.nbytes, face_vertices, GL_STATIC_DRAW)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO[1])
    glBufferData(GL_ELEMENT_ARRAY_BUFFER,face_indices.nbytes, face_indices, GL_STATIC_DRAW)
    #enable vertices
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(0))#从第0个数据开始，元素个数为3
    #enable color
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(12))#从第1个数据开始，元素个数为3
    #设定位置
    face_pos =  pyrr.matrix44.create_from_translation(pyrr.Vector3([0, 0, 0])) #定义物体世界坐标 0, 1, -3
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, face_pos)
    glDrawElements(GL_TRIANGLES, len(face_indices), GL_UNSIGNED_INT, None)
    
    #载入球体
    glBindBuffer(GL_ARRAY_BUFFER, VBO[1])
    glBufferData(GL_ARRAY_BUFFER, sphere_vertices.nbytes, sphere_vertices, GL_STATIC_DRAW)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO[1])
    glBufferData(GL_ELEMENT_ARRAY_BUFFER,sphere_indices.nbytes, sphere_indices, GL_STATIC_DRAW)
    #enable vertices
    glEnableVertexAttribArray(0)
    glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(0))#从第0个数据开始，元素个数为3
    #enable color
    glEnableVertexAttribArray(1)
    glVertexAttribPointer(1, 3, GL_FLOAT, GL_FALSE, 24, ctypes.c_void_p(12))#从第1个数据开始，元素个数为3
    #设定位置
    obj_pos = pyrr.matrix44.create_from_translation(pyrr.Vector3([0.2,0.15,0.8]))
    glUniformMatrix4fv(model_loc, 1, GL_FALSE, obj_pos)
    glDrawElements(GL_TRIANGLES, len(sphere_indices), GL_UNSIGNED_INT, None)
    
    glfw.swap_buffers(window)

# terminate glfw, free up allocated resources
glfw.terminate()